---
title: Overview of the event
---

We are using `PSP_SWP_SPI_SF00_L3_MOM` dataset

In [2]:
#| echo: false
#| output: false
from sunpy.time import TimeRange
from space_analysis.missions.psp.plasma import load_data as load_data_plasma
from space_analysis.utils.speasy import Variables
import space_analysis.missions.dscovr as dscovr

from space_analysis.plasma.formulary import df_beta, df_Alfven_speed
from space_analysis.utils.basic import resample

import polars as pl
from beforerr.polars import pl_norm

import astropy.units as u

from datetime import timedelta

In [3]:
psp_start = '2019-04-07T01:00'
psp_end = '2019-04-07T09:00'

earth_start = '2019-04-09'
earth_end = '2019-04-14'

every = timedelta(minutes=4)

In [4]:
psp_timerange = TimeRange(psp_start, psp_end)
psp_vec_cols = [f"V{_} RTN" for _ in ["x", "y", "z"]]
psp_mag_cols = [f"B{_} SC" for _ in ["x", "y", "z"]]

earth_timerange = TimeRange(earth_start, earth_end)
dscovr_vec_cols = [f"V{_} (GSE)" for _ in ["X", "Y", "Z"]]

In [5]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [6]:
dataset = "PSP_SWP_SPI_SF00_L3_MOM"
parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "MAGF_SC", "SUN_DIST"]

In [7]:
psp_plasma = Variables(
    dataset=dataset, parameters=parameters, timerange=validate(psp_timerange)
).get_data()

## Radial evolution of the coronal hole high-speed streams

@perroneRadialEvolutionSolar2019 used HELIOS observations to study the radial evolution of the solar wind in coronal hole high-speed streams. They found that The radial dependence of the proton number density, np, and the magnetic field, B, is given by

The radial dependence of the proton number density, np is

$$n_p = (2.4 ± 0.1)(R/R_0)^{−(1.96±0.07)} cm^{−3}$$

$$B = (5.7 ± 0.2)(R/R_0)^{−(1.59±0.06)} nT$$

The faster decrease of the magnetic than kinetic pressure is reﬂected in the radial proton plasma beta variation

$$β_p = P_k/P_B = (0.55 ± 0.04)(R/R_0)^{(0.4±0.1)}.$$

The behaviour of the parallel proton plasma beta is similar 

$$β_{‖} = (0.37 ± 0.03)(R/R_0)^{(0.8±0.1)}$$

In [8]:
def plasma_r_evolution(
    df: pl.DataFrame,
    alpha_beta_r = 0.4,
    alpha_beta_parallel_r = 0.9,
    alpha_n = -2,
    alpha_B = -1.63,
    alpha_plasma_speed = 0,
):
    return df.with_columns(
        plasma_speed_1AU = pl.col('plasma_speed') * (1 / pl.col("distance2sun"))**alpha_plasma_speed,
        n_1AU = pl.col('n') * (1 / pl.col("distance2sun"))**alpha_n,
        B_1AU = pl.col('B') * (1 / pl.col("distance2sun"))**alpha_B,
        beta_1AU = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_r,
        beta_parallel = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_parallel_r,
    )

In [9]:
km2au = u.km.to(u.AU)

In [10]:
def process_dscoevr(timerange):
    timerange = validate(timerange)
    # Load DSCOVR magnetic field and plasma data
    dscovr_mag = Variables(
        products=["cda/DSCOVR_H0_MAG/B1F1"], timerange=timerange
    ).get_data()

    dscovr_plasma = Variables(
        dataset="DSCOVR_H1_FC",
        parameters=["Np", "V_GSE", "THERMAL_TEMP"],
        timerange=timerange,
    ).get_data()
    
    dscovr_df = (
        dscovr_plasma.to_polars().pipe(resample, every)
        .sort("time")
        .join_asof(dscovr_mag.to_polars().pipe(resample, every).sort("time"), on="time")
        .with_columns(
            plasma_speed=pl_norm(dscovr_vec_cols),
        )
        .rename(
            {   
                "Ion N": "n",
                "Temperature": "T",
            }
        )
        .collect()
        .pipe(df_beta, T_unit = u.K)
        .pipe(df_Alfven_speed)
        .with_columns(
            plasma_speed_over_Alfven_speed=pl.col("plasma_speed") / pl.col("Alfven_speed"),
        )
    )
    return dscovr_df

In [11]:
from copy import deepcopy

In [12]:
dscovr_df_previous = process_dscoevr(deepcopy(earth_timerange).previous())
dscovr_df = process_dscoevr(earth_timerange)
dscovr_df_next = process_dscoevr(deepcopy(earth_timerange).next())

In [13]:
psp_plasma_r = (
    psp_plasma.to_polars().pipe(resample, every)
    .with_columns(
        B=pl_norm(psp_mag_cols),
        plasma_speed = pl_norm(psp_vec_cols),
        distance2sun = pl.col('Sun Distance') * km2au,
    )
    .rename(
        {
            "Density": "n",
            "Temperature": "T",
        }
    )
    .collect()
    .pipe(df_beta)
    .pipe(plasma_r_evolution)
    .pipe(df_Alfven_speed)
    .pipe(df_Alfven_speed, B = "B_1AU", n = "n_1AU", col_name = "Alfven_speed_1AU")
    .with_columns(
        plasma_speed_over_Alfven_speed = pl.col("plasma_speed") / pl.col("Alfven_speed"),
        plasma_speed_over_Alfven_speed_1AU = pl.col("plasma_speed_1AU") / pl.col("Alfven_speed_1AU"),
    )
)

In [14]:
plot = psp_plasma_r.plot(x="time", y=["plasma_speed", "beta", "Alfven_speed", "plasma_speed_over_Alfven_speed"], subplots=True, shared_axes=False).cols(1)

plot_1AU = psp_plasma_r.plot(x="time", y=["plasma_speed_1AU", "beta_1AU", "Alfven_speed_1AU", "plasma_speed_over_Alfven_speed_1AU"], subplots=True, shared_axes=False).cols(1)

plot + plot_1AU

:Layout
   .NdLayout.I  :NdLayout   [Variable]
      :Curve   [time]   (value)
   .NdLayout.II :NdLayout   [Variable]
      :Curve   [time]   (value)

In [67]:
import hvplot.polars  # noqa
import holoviews as hv
hv.archive.auto()


<IPython.core.display.Javascript object>

Automatic capture is now enabled. [2024-02-04 20:52:05]


In [48]:
hvplot.extension('bokeh', 'matplotlib')
hvplot.output(backend='matplotlib', fig='svg')

In [58]:
def compare_df(df1, df2):
    df1_plot = df1.plot.scatter(
        x="beta", y="plasma_speed_over_Alfven_speed", label="PSP"
    ) * df1.plot.scatter(
        x="beta_1AU",
        y="plasma_speed_over_Alfven_speed_1AU",
        label="PSP (1AU predicted)",
    )

    df2_plot = df2.plot.scatter(
        x="beta",
        y="plasma_speed_over_Alfven_speed",
        label="DSCOVR",
        alpha=0.2,
    )

    xlabel = "Plasma beta"
    # ylabel=r"$v_i$ / $v_A$"
    ylabel = "Plasma speed over Alfven speed"
    title = "Plasma speed over Alfven speed vs plasma beta"

    return (df2_plot * df1_plot).opts(xlabel=xlabel, ylabel=ylabel, logx=True, logy=True)

In [84]:
fig = compare_df(psp_plasma_r, dscovr_df_previous).opts(title="Previous Period")
hvplot.save(fig, "../figures/evolution/plasma-adiabatic-evolution-previous.svg")

In [88]:
fig = compare_df(psp_plasma_r, dscovr_df).opts(title="Current Period")
hvplot.save(fig, "../figures/evolution/plasma-adiabatic-evolution.svg")


In [87]:
fig = compare_df(psp_plasma_r, dscovr_df_next).opts(title="Next Period")
hvplot.save(fig, "../figures/evolution/plasma-adiabatic-evolution-next.svg")